#CNN
## CNN 초기화 인수
- input_dim = 입력 데이터(채널 수, 높이,너비)의 차원
- conv_param = 합성곱 계층의 하이퍼 파라미터
  -filter_num = 필터 수
  - filter_size = 필터 크기
  - stride = 스트라이드
  - pad = 패딩
  - hidden_size = 은닉층 뉴런 수
  - output_size = 출력층 뉴런 수
  - weight_init_std = 초기화 때의 가중치 표준편차

In [7]:
import numpy as np
#from common.util import im2col
#import OrderedDict
#import SoftmaxWithLoss

In [9]:
#reshape 예시
W= np.random.rand(10,3,5,5)
FN,C,FH,FW = W.shape

W = W.reshape(FN,-1).T
print(W.shape)

(75, 10)


In [11]:
class Convolution:
  def __init__(self, W, b, stride=1, pad=0) :
    self.W = W
    self.b = b
    self.stride = stride
    self.pad = pad
  
  def forward(self,x):
    FN, C, FH, FW = self.W.shape #FN:필터개수, C:채널개수, FH,FW: 필터높이,너비
    N, C, H, W = x.shape
    out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
    out_w = int(1 + (W + 2*self.pad - FW) / self.stride)

    col = im2col(x, FH, FW, self.stride, self.pad)
    col_W = self.w.reshape(FN, -1).T #필터 전개(2차원 배열로)
    out = np.dot(col,col_W) + self.b

    out = out.reshape(N, out_h, out_w, -1).transpose(0,3,1,2) # 배열 위치 변경

In [12]:
#풀링 계층 정의
class Pooling:
  def __init__(self, pool_h, pool_w, stride=1, pad=0):
    self.pool_h = pool_h
    self.pool_w = pool_w
    self.stride = stride
    self.pad = pad

  def forward(self,x):
    N, C, H, W = x.shape
    out_h = int(1+(H-self.pool_h)/self.stride)
    out_w = int(1+(W-self.pool_w)/self.stride)

    # 전개(1)
    col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
    col = col.reshape(-1, self.pool_h*self.pool_w)

    # 최대값(2)
    out = np.max(col, axis=1)

    # 성형(3)
    out = out.reshape(N, out_h, out_w, C).transpose(0,3,1,2)

    return out

In [15]:
# 클래스 정의
class SimpleConvNet:
  #합성곱 계층의 하이퍼 파라미터를 딕셔너리에 저장
  def __init__(self, input_dim=(1,28,28),
               conv_param = {'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
               hidden_size=100, output_size=10, weight_init_std=0.001):
    filter_num = conv_param['filter_num']
    filter_size = conv_param['filter_size']
    filter_pad = conv_param['pad']
    filter_stride = conv_param['stride']
    input_size = input_dim[1]
    conv_output_size = (input_size - filter_size +2*filter_pad)/ filter_stride +1
    pool_output_size = int(filter_num*(conv_output_size/2)*(conv_output_size/2))

    
    #가중치 매개변수 초기화
    self.params = {}
    self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
    self.params['b1'] = np.zeros(filter_num)
    self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
    self.params['b2'] = np.zeros(hidden_size)

    #CNN 구성 계층 생성
    self.layers = OrderedDict()
    self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'], conv_param['stride'], conv_param['pad'])
    self.layers['Relu1'] = Relu()
    self.layers['Affine1'] = Affine(self.params['W2'],self.params['b2'])
    self.layers['Relu2'] = Relu()
    self.layers['Affine2'] = Affine(self.params['W3'],self.params['b3'])
    
    self.last_layer = SoftmaxWithLoss()

#predict, loss 메서드
  def predict(self,x):
    for layer in self.layers.values():
      x = layer.forward(x)
    return x
  
  def loss(self,x,t):
    y = self.predict(x)
    return self.lastLayer.forward(y,t)

#오차역전파 기울기 구하기
  def gradient(self, x, t):
    #순전파
    self.loss(x,t)

    #역전파
    dout = 1
    dout = self.lastLayer.backward(dout)

    layers = list(self.layers.values())
    layers.reverse()
    for layer in layers:
      dout = layer.backward(dout)
    
    #결과 저장
    grads= {}
    grads['W1'] = self.layers['Conv1'].dW
    grads['b1'] = self.layers['Conv1'].db
    grads['W2'] = self.layers['Alffine1'].dW
    grads['b2'] = self.layers['Alffine1'].db
    grads['W3'] = self.layers['Alffine2'].dW
    grads['b3'] = self.layers['Alffine2'].db

    return grads

#CNN 기초(https://cs231n.github.io/convolutional-networks/)

- 합성곱 계층의 입출력 데이터를 특징맵(Feature map)이라고도 합니다.
- 합성곱 연산 후 출력데이터 크기는  
OH = {(H + 2*P - FH)/S} + 1  
OW = {(W + 2*P - FW)/S} + 1
(데이터 입력크기:(H,W), 필터 크기 : (FH,FW), 출력 크기 : (OH,OW), 패딩 : P, 스트라이드, S)
*** 해당 연산을 통해 출력 크기가 정수가 아닌 경우에는 가장 가까운 정수로 반올림한다.

